In [5]:
import cv2
import os
import glob
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import RandomFourierFeatures

In [6]:
char_w = 30
char_h = 60

In [7]:
pathTest = '.\\data\\2\\1.jpg'

img = cv2.imread(pathTest, 0)
  
# Displaying the image
cv2.imshow('image', img)
cv2.waitKey(0);
cv2.destroyAllWindows(); 
cv2.waitKey(1)

print(type(img.shape))
print(img.shape)
print(type(np.array(img)))




<class 'tuple'>
(60, 30)
<class 'numpy.ndarray'>


In [10]:
def get_char_data(path):
    
    char_list = []
    label_list = []

    # label digit
    for num_iter in range(10):
        
        # print(num_iter)

        for img_path in glob.iglob(path + str(num_iter) + '/*.jpg'):
            # print(img_path)
            # flags = 0 to take gray scale img
            img = cv2.imread(img_path, 0)
            # convert tuple to np.arr
            img = np.array(img)
            img = img.reshape( -1, char_h * char_w)

            # print(img.shape)

            char_list.append(img)
            label_list.append( int(num_iter)  )

    # label alphabet

    for alpha_iter in range(65, 91):
        # print(alpha_iter)
        for img_path in glob.iglob(path + str(alpha_iter) + '/*.jpg'):
            # print(img_path)

            img = cv2.imread(img_path, 0)
            img = np.array(img)
            img = img.reshape(-1, char_h * char_w)

            # print(img.shape)

            char_list.append(img)
            label_list.append( int(alpha_iter) )

    return char_list, label_list 

In [6]:
# data_path = 'data/'
data_path = 'data2/'
char_list, label_list = get_char_data(data_path)


char_list = np.array(char_list, dtype = np.float32)
label_list = np.array(label_list)

char_list = char_list.reshape(-1, char_h * char_w)
label_list = label_list.reshape(-1, 1)

# print(char_list.shape)
# print(label_list.shape)

svm_model = cv2.ml.SVM_create()
svm_model.setType(cv2.ml.SVM_C_SVC)

svm_model.setKernel(cv2.ml.SVM_INTER)
svm_model.setTermCriteria( (cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-6) )

svm_model.train(char_list, cv2.ml.ROW_SAMPLE, label_list)



svm_model.save('svm2.xml')


0
data2/0\0_1.jpg
(1, 1800)
data2/0\0_10.jpg
(1, 1800)
data2/0\0_100.jpg
(1, 1800)
data2/0\0_1000.jpg
(1, 1800)
data2/0\0_1001.jpg
(1, 1800)
data2/0\0_1002.jpg
(1, 1800)
data2/0\0_1003.jpg
(1, 1800)
data2/0\0_1004.jpg
(1, 1800)
data2/0\0_1005.jpg
(1, 1800)
data2/0\0_1006.jpg
(1, 1800)
data2/0\0_1007.jpg
(1, 1800)
data2/0\0_1008.jpg
(1, 1800)
data2/0\0_1009.jpg
(1, 1800)
data2/0\0_101.jpg
(1, 1800)
data2/0\0_1010.jpg
(1, 1800)
data2/0\0_1011.jpg
(1, 1800)
data2/0\0_1012.jpg
(1, 1800)
data2/0\0_1013.jpg
(1, 1800)
data2/0\0_1014.jpg
(1, 1800)
data2/0\0_1015.jpg
(1, 1800)
data2/0\0_1016.jpg
(1, 1800)
data2/0\0_102.jpg
(1, 1800)
data2/0\0_103.jpg
(1, 1800)
data2/0\0_104.jpg
(1, 1800)
data2/0\0_105.jpg
(1, 1800)
data2/0\0_106.jpg
(1, 1800)
data2/0\0_107.jpg
(1, 1800)
data2/0\0_108.jpg
(1, 1800)
data2/0\0_109.jpg
(1, 1800)
data2/0\0_11.jpg
(1, 1800)
data2/0\0_110.jpg
(1, 1800)
data2/0\0_111.jpg
(1, 1800)
data2/0\0_112.jpg
(1, 1800)
data2/0\0_113.jpg
(1, 1800)
data2/0\0_114.jpg
(1, 1800)
data2

# Accuracy, Precision, Recall, F-Score,... for 2 Class:
<img height="50%" src = "https://raw.githubusercontent.com/thangnch/photos/master/Screen%20Shot%202020-06-16%20at%2014.00.30.png">

## Accuracy, Precision, Recall, F-Score,.. for Multi Class:

- [PDF](http://rali.iro.umontreal.ca/rali/sites/default/files/publis/SokolovaLapalme-JIPM09.pdf)

In [18]:
def get_presision_and_recall(model_path, data_pred_path):
    
    svm_model = cv2.ml.SVM_load(model_path)
    char_list, label_list = get_char_data(data_pred_path)
    

    total_Precision = 0
    total_Recall = 0
    
    unique_Label = set(label_list)

    step = 0
    total_step = len(unique_Label) * len(char_list)
    # print(label_list)
    # print(unique_Label)

    for iter in  unique_Label :
        true_Positive = 0
        false_Positive = 0
        true_Negative = 0
        false_Negative = 0

        single_Precision = 0
        single_Recall = 0

        for j in range( len(char_list)):
            
            print(str(step) + "/" + str(total_step) )
            step += 1

            char_list[j] = np.array(char_list[j], dtype = np.float32)
            # print(char_list[j] )
            # print(char_list[j].shape)


            pred = svm_model.predict( char_list[j] )[1]
            pred = pred[0][0]
            if( pred == iter and iter  == label_list[j] ):
                true_Positive +=1
            elif( pred != iter and iter != label_list[j] ):
                true_Negative +=1
            elif( pred == iter and iter != label_list[j]):
                false_Positive +=1
            else:
                false_Negative +=1
            
        single_Precision = true_Positive / (true_Positive + false_Positive)
        single_Recall = true_Positive / (true_Positive + false_Negative)

        total_Precision += single_Precision
        total_Recall += single_Recall
    
    return total_Precision / len(unique_Label), total_Recall / len(unique_Label) 
        

        
def get_Average_Accuracy_and_Error_rate(model_path, data_pred_path):

    svm_model = cv2.ml.SVM_load(model_path)
    char_list, label_list = get_char_data(data_pred_path)

    
    total_Binary_Accuracy = 0
    total_Error_rate = 0
    
    unique_Label = set(label_list)

    # print(label_list)
    # print(unique_Label)
    step = 0
    total_step = len(unique_Label) * len(char_list)
    
    for iter in  unique_Label :
        true_Positive = 0
        true_Negative = 0
        Binary_Accuracy = 0
        single_Error_rate = 0

        for j in range( len(char_list)):
            
            print(str(step) + "/" + str(total_step) )
            step += 1
            
            char_list[j] = np.array(char_list[j], dtype = np.float32)
            # print(char_list[j] )
            # print(char_list[j].shape)


            pred = svm_model.predict( char_list[j] )[1]
            pred = pred[0][0]
            if( pred == iter and iter  == label_list[j] ):
                true_Positive +=1
            elif( pred != iter and iter != label_list[j] ):
                true_Negative +=1
            else:
                single_Error_rate += 1
            
        Binary_Accuracy = (true_Positive + true_Negative) / len(char_list)
        total_Binary_Accuracy += Binary_Accuracy

        single_Error_rate /= len(char_list)
        total_Error_rate += single_Error_rate

    return total_Binary_Accuracy / len( unique_Label ), total_Error_rate / len(unique_Label)            

def get_F_Score(precision, recall, beta):
    
    return (beta * beta + 1) * (precision * recall) / (beta * beta * precision + recall)


In [20]:
model_path = 'svm2.xml'
data_pred_path = 'data/' 
a_e = get_Average_Accuracy_and_Error_rate(model_path, data_pred_path)
p_rc = get_presision_and_recall(model_path, data_pred_path)
beta = 1
fs = get_F_Score(p_rc[0], p_rc[1],beta)

print("Accuracy: "   + str( a_e[0] ) )
print("Error rate: " + str( a_e[1] ) )
print("Precision: "  + str( p_rc[0] ) )
print("Recall: "     + str( p_rc[1] ) )
print(f"F-{beta} Score: "  + str( fs )      )

os.chdir('E:\Sem5\AIP391\src\Character_detection_with_SVM')
fh = open('Confusion_Matrix.txt', "w+")
fh = open('Confusion_Matrix.txt', "a+")

fh.write("Accuracy: "   + str( a_e[0] ))
fh.write("Error rate: " + str( a_e[1] ) )
fh.write("Precision: "  + str( p_rc[0] ) )
fh.write("Recall: "     + str( p_rc[1] ) )
fh.write("F-"+str(beta)+": "  + str( fs )      )

0/32457
1/32457
2/32457
3/32457
4/32457
5/32457
6/32457
7/32457
8/32457
9/32457
10/32457
11/32457
12/32457
13/32457
14/32457
15/32457
16/32457
17/32457
18/32457
19/32457
20/32457
21/32457
22/32457
23/32457
24/32457
25/32457
26/32457
27/32457
28/32457
29/32457
30/32457
31/32457
32/32457
33/32457
34/32457
35/32457
36/32457
37/32457
38/32457
39/32457
40/32457
41/32457
42/32457
43/32457
44/32457
45/32457
46/32457
47/32457
48/32457
49/32457
50/32457
51/32457
52/32457
53/32457
54/32457
55/32457
56/32457
57/32457
58/32457
59/32457
60/32457
61/32457
62/32457
63/32457
64/32457
65/32457
66/32457
67/32457
68/32457
69/32457
70/32457
71/32457
72/32457
73/32457
74/32457
75/32457
76/32457
77/32457
78/32457
79/32457
80/32457
81/32457
82/32457
83/32457
84/32457
85/32457
86/32457
87/32457
88/32457
89/32457
90/32457
91/32457
92/32457
93/32457
94/32457
95/32457
96/32457
97/32457
98/32457
99/32457
100/32457
101/32457
102/32457
103/32457
104/32457
105/32457
106/32457
107/32457
108/32457
109/32457
110/32457


23

In [60]:
# test predict
import cv2
import numpy as np

SVM_mol = cv2.ml.SVM_load('svm2.xml')
# print(SVM_mol.isTrained)
# print(SVM_mol.isClassifier)


img_test_path = r'test_img/3.jpg'

img_test = cv2.imread(img_test_path, 0)

cv2.imshow('GrayScale', img_test)
cv2.waitKey();

img_test = cv2.resize(img_test, dsize = ( char_w, char_h) ) 

img_test = np.array(img_test, dtype = np.float32)
print(img_test)
print(img_test.shape)

cv2.imshow('After resize', img_test)
cv2.waitKey();



# img_test = 
# print(img_test.shape)

img_test = img_test.reshape(-1, char_w * char_h)

print(img_test)
print(img_test.shape)


print(SVM_mol.predict(img_test))
pred = SVM_mol.predict(img_test)[1]

pred = pred[0][0]

print('predict value: ',end= '')
if pred < 10:
    print(pred)
else: 
    print(chr(pred))






[[  1.   0.   1. ...   0.   1.   1.]
 [  1.   1. 169. ... 170.   0.   1.]
 [  1. 102. 255. ... 254.   0.   0.]
 ...
 [  1. 203. 255. ... 255. 255.  67.]
 [  1.   1. 169. ... 170.   1.   1.]
 [  0.   1.   0. ...   1.   1.   1.]]
(60, 30)
[[1. 0. 1. ... 1. 1. 1.]]
(1, 1800)
(0.0, array([[88.]], dtype=float32))
predict value: X
